In [37]:
import numpy as np

dtype = {
    "k": np.float64,
    "a": 555,
    "b": 666,
}

for each in dtype:
    print(each)

a = np.zeros(20, dtype=np.float32)
a.astype(np.float64)



k
a
b


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])

In [20]:
import urllib
import gzip
import os
import numpy as np

def download_minist():
    urls = {
        "train_images": "http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz",
        "train_labels": "http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-labels-idx1-ubyte.gz",
        "test_images": "http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-images-idx3-ubyte.gz",
        "test_labels": "http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-labels-idx1-ubyte.gz"
    }

    download = not os.path.exists('data')

    if download:
        os.makedirs('data')

    train_images = download_and_extract_data(urls["train_images"], os.path.join("data", "train-images-idx3-ubyte.gz"), download)
    train_labels = download_and_extract_data(urls["train_labels"], os.path.join("data", "train-labels-idx1-ubyte.gz"), download)
    test_images = download_and_extract_data(urls["test_images"], os.path.join("data", "t10k-images-idx3-ubyte.gz"), download)
    test_labels = download_and_extract_data(urls["test_labels"], os.path.join("data", "t10k-labels-idx1-ubyte.gz"), download)

    train_images = train_images.reshape(-1, 28 * 28)
    test_images = test_images.reshape(-1, 28 * 28)

    data = {
        'train_images': train_images,
        'train_labels': train_labels,
        'test_images': test_images,
        'test_labels': test_labels
    }

    return data

def download_and_extract_data(url, file_name, download=True):
    if download:
        urllib.request.urlretrieve(url, file_name)

    with gzip.open(file_name, 'rb') as f:
        if 'images' in file_name:
            data = np.frombuffer(f.read(), dtype=np.uint8, offset=16)
            print(data.shape)
            data = data.reshape((-1, 28, 28))
            print(data.shape)
        else:
            data = np.frombuffer(f.read(), dtype=np.uint8, offset=8)
    return data


data = download_minist()

print(data['train_images'][0].shape)


(47040000,)
(60000, 28, 28)
(7840000,)
(10000, 28, 28)
(784,)


usage: ipykernel_launcher.py [-h] --values VALUES [VALUES ...]
ipykernel_launcher.py: error: the following arguments are required: --values


SystemExit: 2

c:\Users\雪浪\AppData\Local\Programs\Python\Python311\Lib\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
